In [ ]:
# Upload the packages
import os
import ee
import geemap

In [ ]:
# Create an ipyleaflet-based interactive map with Geemap
Map = geemap.Map()
Map

In [ ]:
# Draw shapes on the interactive map using the Drawing tools & use it as ROI 
# Draw any shapes on the map using the Drawing tools before executing this code block
feature = Map.draw_last_feature
roi = feature.geometry()

In [ ]:
# Upload a collection of images (Sentinel-5p, Ground lev SO2) & visualizing it
# Sentinel 5-p SO2 vertical column density at ground level (DOAS technique)
collection = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_SO2')\
.select('SO2_column_number_density')\
.filterDate('2020-03-01', '2020-12-31')\
.filterBounds(roi)

# Define the range
val_max = 0.0
val_min = 0.0005
band_viz = {
  'min': val_min,
  'max': val_max,
  'opacity': 1.0,
  'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

Map.addLayer(collection.median().clip(roi), band_viz, 'groLev_SO2_2020')

In [ ]:
# Reduce the collection with a median reducer.
collection_median = collection.reduce(ee.Reducer.median())
# Clip the imageCollection 
collection_median_clip = collection_median.clip(roi).unmask()
# Visualize the clipped image
Map.addLayer(collection_median_clip, band_viz, 'SO2_2020')

In [ ]:
# Select the path & define the filename
path = '/Users/...'
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'S5p.tif')

In [ ]:
# Export the image
geemap.ee_export_image(collection_median_clip, 
                       filename=filename, 
                       scale=1000, 
                       region=roi, 
                       file_per_band=True)